# Importing Libraries

In [57]:
import sys
import pandas as pd
import numpy as np
import math
import plotly.graph_objects as go
import plotly
from pmdarima import auto_arima
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Ignoring warnings
import warnings
warnings.filterwarnings("ignore")

# To import the main.py file
sys.path.append('../')
from python_files import main

In [58]:
%matplotlib inline
plotly.offline.init_notebook_mode(connected=True)

# Data Preprocessing Functions

In [59]:
def get_data():
    confirmed_global, deaths_global, recovered_global, country_cases = main.collect_data()
    
    recovered = recovered_global.groupby("country").sum().T
    deaths = deaths_global.groupby("country").sum().T
    confirmed = confirmed_global.groupby("country").sum().T
    
    deaths.index = pd.to_datetime(deaths.index, infer_datetime_format = True)
    recovered.index = pd.to_datetime(recovered.index, infer_datetime_format = True)
    confirmed.index = pd.to_datetime(confirmed.index, infer_datetime_format = True)
    
    return deaths, recovered, confirmed

In [60]:
def create_data_frame(dataframe,country):
    deaths, recovered, confirmed = get_data()
    if dataframe == 'deaths':
        data = pd.DataFrame(index = deaths.index, data = deaths[country].values, columns = ["Total"])

    elif dataframe == 'recovered':
        data = pd.DataFrame(index = recovered.index, data = recovered[country].values, columns = ["Total"])

    elif dataframe == 'confirmed':
        data = pd.DataFrame(index = confirmed.index, data = confirmed[country].values, columns = ["Total"])

    data = data[(data != 0).all(1)]
    
    data['Date'] = data.index
    cols = [data.columns[-1]] + [col for col in data if col != data.columns[-1]]
    data = data[cols]   

    return data

# Graphing Functions

In [61]:
def plot_forecast(data,forecast):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=data["Date"], y=data["Total"],   
                            mode='lines',
                            name='Up till now '))
        
    fig.add_trace(go.Scatter(x=forecast.index, y=forecast["cumulative"],   
                            mode='lines',
                            name='Prediction*'))

    fig.update_layout(title={
                'text': "Forecasted results",
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                        template = "plotly_dark",
                        xaxis_title="Date",
                        yaxis_title="Cases",
                        legend_title="Legend ",
                        font=dict(
                                family="Arial",
                                size=15,
                                color="white"
                                )
                        )
    return fig

# Functions to Train and Test the Model

In [62]:
def find_params(train_set):
    stepwise_model = auto_arima(train_set, start_p = 0, start_q = 0,
                               max_p = 1, max_q = 1, m = 7,
                               start_P = 0, max_P=1, start_Q=0, max_Q=1, seasonal = True,
                               d = None, D = 1, n_jobs=-1, trace = True,
                               error_action = 'ignore',  
                               suppress_warnings = True, 
                               stepwise = True)
    return stepwise_model

In [63]:
def Predict(stepwise_model,train,test):
    
    stepwise_model.fit(train)
    
    pred = stepwise_model.predict(n_periods=len(test))

    
    pred = pd.DataFrame(pred,index = test.index,columns=['Prediction'])
    pred =  pred.values.flatten()
   
    return pred

# Error Function

In [64]:
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Function for Forecasting

In [65]:
def Future(order,seasonal_order,train,test,data):
    
    model = SARIMAX(data.diff()["Total"][1:],  
                        order = order,  
                        seasonal_order = seasonal_order) 
    result = model.fit() 
  
    forecast = result.predict(start = len(data),  
                          end = (len(data)-1) + 14).rename('Forecast') 
    forecast_cumulative = []
    start = data["Total"][-1]
    for i in forecast.values :
        start = start + i
        forecast_cumulative.append(start)
    forecast["cumulative"] = forecast_cumulative

    

    graph = plot_forecast(data,forecast)

    return forecast_cumulative,graph

# Calling Function

In [66]:
def arima_predict(df_name,country):
    data = create_data_frame(df_name,country)
    
    train = data["Total"][:len(data)*4//5].diff()[1:]
    test = data["Total"][len(data)*4//5:].diff()[1:]
    start = data["Total"][-len(test)-1]
    model = find_params(train)
    pred = Predict(model,train,test)
    predictions_cumulative=[]
    for i in pred :
        start = start + i
        predictions_cumulative.append(start)
    y_test_cumulative = data["Total"][-len(test):]

    MAPE = mape(y_test_cumulative,predictions_cumulative)

    order=model.get_params()['order']
    seasonal_order=model.get_params()['seasonal_order']
    
    forecast,graph = Future(order, seasonal_order, train, test,data)
    return forecast,graph,MAPE

# Example

In [67]:
forecast,graph,error = arima_predict("confirmed","Iceland")

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[7] intercept   : AIC=1342.195, Time=0.01 sec
 ARIMA(1,0,0)(1,1,0)[7] intercept   : AIC=1286.214, Time=0.18 sec
 ARIMA(0,0,1)(0,1,1)[7] intercept   : AIC=1311.283, Time=0.18 sec
 ARIMA(0,0,0)(0,1,0)[7]             : AIC=1340.197, Time=0.01 sec
 ARIMA(1,0,0)(0,1,0)[7] intercept   : AIC=1301.055, Time=0.04 sec
 ARIMA(1,0,0)(1,1,1)[7] intercept   : AIC=inf, Time=0.54 sec
 ARIMA(1,0,0)(0,1,1)[7] intercept   : AIC=inf, Time=0.32 sec
 ARIMA(0,0,0)(1,1,0)[7] intercept   : AIC=1343.203, Time=0.23 sec
 ARIMA(1,0,1)(1,1,0)[7] intercept   : AIC=1255.080, Time=0.30 sec
 ARIMA(1,0,1)(0,1,0)[7] intercept   : AIC=1276.406, Time=0.09 sec
 ARIMA(1,0,1)(1,1,1)[7] intercept   : AIC=inf, Time=0.63 sec
 ARIMA(1,0,1)(0,1,1)[7] intercept   : AIC=inf, Time=0.67 sec
 ARIMA(0,0,1)(1,1,0)[7] intercept   : AIC=1312.788, Time=0.16 sec
 ARIMA(1,0,1)(1,1,0)[7]             : AIC=1253.090, Time=0.14 sec
 ARIMA(1,0,1)(0,1,0)[7]             : AIC=1274.406, T

In [68]:
print("PREDICTIONS FOR THE NEXT 14 DAYS : \n")
for i in range(len(forecast)):
    print(int(forecast[i]))

PREDICTIONS FOR THE NEXT 14 DAYS : 

2330
2356
2382
2412
2473
2487
2512
2539
2571
2603
2640
2715
2731
2761


In [69]:
error

0.8812787191475635

In [70]:
graph.show()